In [ ]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    RobertaForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/test.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function(example):
    combined_input = f"aspekt: {example['aspect']} text: {example['text']}"

    encoding = tokenizer(
        combined_input,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Map:   0%|          | 0/329 [00:00<?, ? examples/s]

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=3
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.085815,0.413110
2,1.062600,0.807007,0.667683
3,0.835000,0.752579,0.679878
4,0.835000,0.772361,0.705793
5,0.611900,0.830793,0.713415
6,0.490200,0.797852,0.710366
7,0.380600,0.886505,0.708841
8,0.380600,0.987310,0.721037
9,0.276200,1.043670,0.719512
10,0.237500,1.070404,0.714939


TrainOutput(global_step=1440, training_loss=0.5477259880966611, metrics={'train_runtime': 1224.9448, 'train_samples_per_second': 18.744, 'train_steps_per_second': 1.176, 'total_flos': 3020542035517440.0, 'train_loss': 0.5477259880966611, 'epoch': 10.0})

In [ ]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 0.9873102903366089, 'eval_accuracy': 0.7210365853658537, 'eval_runtime': 8.7171, 'eval_samples_per_second': 75.255, 'eval_steps_per_second': 4.703, 'epoch': 10.0}


In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.3045177459716797, 'eval_accuracy': 0.6382978723404256, 'eval_runtime': 4.7242, 'eval_samples_per_second': 69.642, 'eval_steps_per_second': 4.445, 'epoch': 10.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base")

('/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base/vocab.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base/merges.txt',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base/added_tokens.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base/tokenizer.json')

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/roberta-base")

## Vyhodnocovací metriky

In [ ]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [ ]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.3045177459716797, 'eval_model_preparation_time': 0.0032, 'eval_accuracy': 0.6382978723404256, 'eval_runtime': 5.0348, 'eval_samples_per_second': 65.346, 'eval_steps_per_second': 8.342}


In [ ]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [ ]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.3045177459716797, 'eval_model_preparation_time': 0.0045, 'eval_f1': 0.6363427668616188, 'eval_runtime': 5.1495, 'eval_samples_per_second': 63.89, 'eval_steps_per_second': 8.156}


In [ ]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [ ]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.3045177459716797, 'eval_model_preparation_time': 0.0118, 'eval_f1': 0.6298264536063686, 'eval_runtime': 5.2599, 'eval_samples_per_second': 62.548, 'eval_steps_per_second': 7.985}


## Praktické testy

In [ ]:
def predict_sentiment(model, tokenizer, aspect, text):
    combined_input = f"aspekt: {aspect} text: {text}"

    encoding = tokenizer(
        text=combined_input,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )
    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [ ]:
example_aspect = "výkon"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "displej"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "baterie"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-2.1338, -0.5203,  2.7216]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 0.4922, -0.2239, -0.7486]])
Predicted sentiment for L2: negative, expected: negative
tensor([[-0.8187, -0.8156,  1.4364]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [ ]:
example_aspect = "personál"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "stůl"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "jídlo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "pivo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 0.9102, -1.1201,  0.4164]])
Predicted sentiment for R1: negative, expected: negative
tensor([[ 0.8804, -1.0588,  0.3315]])
Predicted sentiment for R2: negative, expected: negative
tensor([[-0.8360, -1.2743,  2.3884]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-0.6666, -1.2402,  2.2463]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [ ]:
example_aspect = "Ósace"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[-0.8936, -0.5864,  1.4433]])
Predicted sentiment for M1: positive, expected: neutral
tensor([[-1.3642, -0.6419,  2.0705]])
Predicted sentiment for M2: positive, expected: positive
tensor([[-0.9535, -0.5944,  1.5495]])
Predicted sentiment for M3: positive, expected: negative


### Sada O

In [ ]:
example_aspect = "den"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "počasí"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-1.9639, -0.6400,  2.7521]])
Predicted sentiment for O1: positive, expected: positive
tensor([[-2.2557, -0.6085,  2.9968]])
Predicted sentiment for O2: positive, expected: negative
